# Road signs Classification

This notebook demonstrates how to classify images from Imagery Insights Road signs dataset using the Gemini 2.5 Flash model via Google Cloud's Vertex AI.

## Install Required Libraries

In [ ]:
%pip install --upgrade google-cloud-bigquery google-cloud-aiplatform

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID and Region.

In [5]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "id": "61d1bfb1",
      "metadata": {
        "id": "61d1bfb1"
      },
      "source": [
        "# Road signs Classification\n",
        "\n",
        "This notebook demonstrates how to classify images from Imagery Insights Road signs dataset using the Gemini 2.5 Flash model via Google Cloud's Vertex AI."
      ]
    },
    {
      "cell_type": "markdown",
      "id": "ab074d90",
      "metadata": {
        "id": "ab074d90"
      },
      "source": [
        "## Install Required Libraries"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "23f28877",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "23f28877",
        "outputId": "57064669-21f3-462c-acf1-bdd4d672a334"
      },
      "outputs": [],
      "source": [
        "%pip install --upgrade google-cloud-bigquery google-cloud-aiplatform"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "de710bd6",
      "metadata": {
        "id": "de710bd6"
      },
      "source": [
        "## Configuration\n",
        "\n",
        "**Important**: Replace the placeholder values below with your actual GCP Project ID and Region."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 12,
      "id": "eefe0d49",
      "metadata": {
        "id": "eefe0d49"
      },
      "outputs": [],
      "source": [
        "PROJECT_ID = ''  # @param {type:\"string\"}\n",
        "REGION = 'us-central1'      # @param {type:\"string\"}\n",
        "\n",
        "# BigQuery Configuration\n",
        "BIGQUERY_DATASET_ID = '' # @param {type:\"string\"}\n",
        "BIGQUERY_TABLE_ID = '' # @param {type:\"string\"}\n",
        "QUERY_LIMIT = 10 # @param {type:\"integer\"}\n",
        "ASSET_TYPE = \"ASSET_CLASS_ROAD_SIGN\" # @param {type:\"string\"}\n",
        "MODEL = \"gemini-2.5-flash\" # @param {type:\"string\"}"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "64efff2f",
      "metadata": {
        "id": "64efff2f"
      },
      "source": [
        "## Imports and Vertex AI Initialization"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 13,
      "id": "70c48cc9",
      "metadata": {
        "id": "70c48cc9"
      },
      "outputs": [],
      "source": [
        "import vertexai\n",
        "from google.cloud import bigquery\n",
        "from google import genai\n",
        "from google.genai.types import Content, Part\n",
        "\n",
        "# Initialize Vertex AI SDK\n",
        "vertexai.init(project=PROJECT_ID, location=REGION)"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "PgTwgtjkjvLO",
      "metadata": {
        "id": "PgTwgtjkjvLO"
      },
      "source": [
        "## Create vertex client"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 14,
      "id": "9lzdGNEMjtC4",
      "metadata": {
        "id": "9lzdGNEMjtC4"
      },
      "outputs": [],
      "source": [
        "client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "ce2a0696",
      "metadata": {
        "id": "ce2a0696"
      },
      "source": [
        "## Fetch Image URIs from BigQuery\n",
        "\n",
        "Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 16,
      "id": "-Mmobxt0kLJB",
      "metadata": {
        "id": "-Mmobxt0kLJB"
      },
      "outputs": [],
      "source": [
        "PROMPT = \"\"\"You will be provided with a photo of a utility pole:\n",
        "{photo_of_utility_pole}\n",
        "\n",
        "You are an image analysis expert. Your task is to analyze an image of a road scene and identify any road signs from a predefined list.\n",
        "InstructionsAnalyze the Image: Carefully analyze the provided image. Your entire analysis MUST be based exclusively on the image provided. Do not infer information or make assumptions beyond what is visually present.\n",
        "Identify Signs: Scan the image to identify all road signs. For each potential sign, use the following logic:\n",
        "a. First, identify its shape and color.\n",
        "b. Next, read any visible text or interpret symbols.\n",
        "c. Finally, match this combination of features to a category in the 'Sign Categories & Features' list below.\n",
        "Format the Output: Present your findings as a JSON array. Each object in the array represents one identified sign and must use the schema defined in the 'JSON Output Schema' section.\n",
        "DefinitionsSign Categories & Features:\n",
        "Stop: Octagonal (8-sided) shape, red background, white text ('STOP').\n",
        "Yield: Downward-pointing triangle shape, white background with a red border.\n",
        "Speed Limit: Vertical rectangle shape, typically white background with black numbers.\n",
        "Pedestrian Crossing: Pentagon (5-sided) shape or square, typically yellow or fluorescent green/yellow with black symbols of people walking.\n",
        "No Parking: Typically a rectangle or square, often with a red circle and slash over the letter 'P'.\n",
        "Turn: Rectangle shape, typically with a black arrow indicating the direction of the turn (left, right, U-turn) on a white or yellow background.\n",
        "Do not enter: A red circle with a white horizontal bar inside.\n",
        "Street name: A horizontal rectangle, typically with a green, blue, or white background and white or black text.\n",
        "Other: Any sign not matching the specific descriptions above.\n",
        "Confidence Level Definitions:\n",
        "High: The sign is clearly visible and legible in the image.\n",
        "Medium: The sign's characteristic shape and/or color are visible, but the text or symbol is unreadable or obscured. The identification is based on shape and color context (e.g., a red octagon is identified as a Stop sign even if the text is unreadable).\n",
        "JSON Output SchemaYour output must be a valid JSON array where each object represents a single sign and contains the following keys:\n",
        "\\\"Sign\\\": [Name of the sign from the 'Sign Categories & Features' list]\n",
        "\\\"Reasoning\\\": [A brief explanation of the visual evidence (shape, color, text).]\n",
        "\\\"Confidence\\\": [\\\"High\\\" or \\\"Medium\\\"]\n",
        "ExampleThis is an illustrative example. Your analysis must be based on the image you are provided.\n",
        "[\n",
        "  {\n",
        "    \\\"Sign\\\": \\\"Stop\\\",\n",
        "    \\\"Reasoning\\\": \\\"Identified a red, octagonal sign with the word \\\\\\\"STOP\\\\\\\" in clear, white text.\\\",\n",
        "    \\\"Confidence\\\": \\\"High\\\"\n",
        "  },\n",
        "  {\n",
        "    \\\"Sign\\\": \\\"Speed Limit\\\",\n",
        "    \\\"Reasoning\\\": \\\"Identified a vertical rectangular sign with a white background. The numbers '45' are legible, consistent with a speed limit sign.\\\",\n",
        "    \\\"Confidence\\\": \\\"High\\\"\n",
        "  },\n",
        "  {\n",
        "    \\\"Sign\\\": \\\"Street name\\\",\n",
        "    \\\"Reasoning\\\": \\\"Identified a green, horizontal rectangular sign. The text is blurry, but the shape and color are characteristic of a street name sign.\\\",\n",
        "    \\\"Confidence\\\": \\\"Medium\\\"\n",
        "  }\n",
        "]\n",
        "\"\"\""
      ]
    },
    {
      "cell_type": "markdown",
      "id": "b05f2cff",
      "metadata": {
        "id": "b05f2cff"
      },
      "source": [
        "## Define Image Classification Function\n",
        "\n",
        "This function takes a GCS URI and a prompt, then uses the Gemini 2.5 Flash model to generate a description of the image."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 9,
      "id": "126856a8",
      "metadata": {
        "id": "126856a8"
      },
      "outputs": [],
      "source": [
        "def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:\n",
        "    \"\"\"\n",
        "    Classifies an image using the Gemini 2.5 Flash model by directly passing its GCS URI.\n",
        "    \"\"\"\n",
        "    try:\n",
        "          contents = [\n",
        "          PROMPT,\n",
        "          Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]\n",
        "\n",
        "          response = client.models.generate_content(model=MODEL, contents=contents)\n",
        "          return response.text\n",
        "    except Exception as e:\n",
        "        print(f\"Error classifying image from URI {gcs_uri}: {e}\")\n",
        "        return \"Classification failed.\""
      ]
    },
    {
      "cell_type": "markdown",
      "id": "Y2TdMKAi4xBJ",
      "metadata": {
        "id": "Y2TdMKAi4xBJ"
      },
      "source": [
        "#Get the image URIs from BigQuery"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "df6a38c6",
      "metadata": {
        "id": "df6a38c6"
      },
      "outputs": [],
      "source": [
        "from google.cloud import bigquery\n",
        "\n",
        "BIGQUERY_SQL_QUERY = f\"\"\"\n",
        "SELECT\n",
        "  *\n",
        "FROM\n",
        "  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}`\n",
        "  WHERE asset_type = '{ASSET_TYPE}'\n",
        "\n",
        "LIMIT {QUERY_LIMIT};\n",
        "\"\"\"\n",
        "\n",
        "# Execute BigQuery Query\n",
        "try:\n",
        "    bigquery_client = bigquery.Client(project=PROJECT_ID)\n",
        "    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)\n",
        "    query_response_data = [dict(row) for row in query_job]\n",
        "    gcs_uris = [item.get(\"gcs_uri\") for item in query_response_data if item.get(\"gcs_uri\")]\n",
        "\n",
        "    print(f\"Successfully fetched {len(gcs_uris)} GCS URIs:\")\n",
        "    for uri in gcs_uris:\n",
        "        print(uri)\n",
        "except Exception as e:\n",
        "    print(f\"An error occurred while querying BigQuery: {e}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "1ae4daad",
      "metadata": {
        "id": "1ae4daad"
      },
      "source": [
        "## Classify Images\n",
        "\n",
        "Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "85b3e0de",
      "metadata": {
        "id": "85b3e0de"
      },
      "outputs": [],
      "source": [
        "\n",
        "if 'gcs_uris' in locals() and gcs_uris:\n",
        "    for uri in gcs_uris:\n",
        "        print(f\"--- Classifying {uri} ---\")\n",
        "        classification = classify_image_with_gemini(uri, PROMPT)\n",
        "        print(f\"Result: {classification}\\n\")\n",
        "else:\n",
        "    print(\"No GCS URIs were found to classify.\")"
      ]
    }
  ],
  "metadata": {
    "colab": {
      "name": "Road_signs_classification.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.10.10"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}


NameError: name 'null' is not defined

## Imports and Vertex AI Initialization

In [13]:
import vertexai
from google.cloud import bigquery
from google import genai
from google.genai.types import Content, Part

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Create vertex client

In [14]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)


## Fetch Image URIs from BigQuery

Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify.

In [16]:
PROMPT = """You will be provided with a photo of a utility pole:
{photo_of_utility_pole}

You are an image analysis expert. Your task is to analyze an image of a road scene and identify any road signs from a predefined list.
InstructionsAnalyze the Image: Carefully analyze the provided image. Your entire analysis MUST be based exclusively on the image provided. Do not infer information or make assumptions beyond what is visually present.
Identify Signs: Scan the image to identify all road signs. For each potential sign, use the following logic:
a. First, identify its shape and color.
b. Next, read any visible text or interpret symbols.
c. Finally, match this combination of features to a category in the 'Sign Categories & Features' list below.
Format the Output: Present your findings as a JSON array. Each object in the array represents one identified sign and must use the schema defined in the 'JSON Output Schema' section.
DefinitionsSign Categories & Features:
Stop: Octagonal (8-sided) shape, red background, white text ('STOP').
Yield: Downward-pointing triangle shape, white background with a red border.
Speed Limit: Vertical rectangle shape, typically white background with black numbers.
Pedestrian Crossing: Pentagon (5-sided) shape or square, typically yellow or fluorescent green/yellow with black symbols of people walking.
No Parking: Typically a rectangle or square, often with a red circle and slash over the letter 'P'.
Turn: Rectangle shape, typically with a black arrow indicating the direction of the turn (left, right, U-turn) on a white or yellow background.
Do not enter: A red circle with a white horizontal bar inside.
Street name: A horizontal rectangle, typically with a green, blue, or white background and white or black text.
Other: Any sign not matching the specific descriptions above.
Confidence Level Definitions:
High: The sign is clearly visible and legible in the image.
Medium: The sign's characteristic shape and/or color are visible, but the text or symbol is unreadable or obscured. The identification is based on shape and color context (e.g., a red octagon is identified as a Stop sign even if the text is unreadable).
JSON Output SchemaYour output must be a valid JSON array where each object represents a single sign and contains the following keys:
\"Sign\": [Name of the sign from the 'Sign Categories & Features' list]
\"Reasoning\": [A brief explanation of the visual evidence (shape, color, text).]
\"Confidence\": [\"High\" or \"Medium\"]
ExampleThis is an illustrative example. Your analysis must be based on the image you are provided.
[
  {
    \"Sign\": \"Stop\",
    \"Reasoning\": \"Identified a red, octagonal sign with the word \\\"STOP\\\" in clear, white text.\",
    \"Confidence\": \"High\"
  },
  {
    \"Sign\": \"Speed Limit\",
    \"Reasoning\": \"Identified a vertical rectangular sign with a white background. The numbers '45' are legible, consistent with a speed limit sign.\",
    \"Confidence\": \"High\"
  },
  {
    \"Sign\": \"Street name\",
    \"Reasoning\": \"Identified a green, horizontal rectangular sign. The text is blurry, but the shape and color are characteristic of a street name sign.\",
    \"Confidence\": \"Medium\"
  }
]
"""

## Define Image Classification Function

This function takes a GCS URI and a prompt, then uses the Gemini 2.5 Flash model to generate a description of the image.

In [9]:
def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:
    """
    Classifies an image using the Gemini 2.5 Flash model by directly passing its GCS URI.
    """
    try:
          contents = [
          PROMPT,
          Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

          response = client.models.generate_content(model=MODEL, contents=contents)
          return response.text
    except Exception as e:
        print(f"Error classifying image from URI {gcs_uri}: {e}")
        return "Classification failed."

#Get the image URIs from BigQuery

In [ ]:
from google.cloud import bigquery

BIGQUERY_SQL_QUERY = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}`
  WHERE asset_type = '{ASSET_TYPE}'

LIMIT {QUERY_LIMIT};
"""

# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]

    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Classify Images

Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt.

In [ ]:

if 'gcs_uris' in locals() and gcs_uris:
    for uri in gcs_uris:
        print(f"--- Classifying {uri} ---")
        classification = classify_image_with_gemini(uri, PROMPT)
        print(f"Result: {classification}\n")
else:
    print("No GCS URIs were found to classify.")